# Análisis de redes Áncash - Pasco

## 1. Parámetros y utilidades de limpieza

In [1]:
from pathlib import Path
import pandas as pd, numpy as np, networkx as nx, json, re

# ----- Parámetros -----
RUTA_ARCHIVO = "contratos.xlsx"      # <-- tu nuevo archivo
HOJA_EXCEL = 0
REGIONES_OBJ = ["ANCASH", "PASCO"]   # regiones que quieres incluir
ANIO_MIN, ANIO_MAX = 2005, 2024
PRORRATEAR_MONTO_CONSORCIO = False
SALIDA = Path("docs")                # para GitHub Pages


# ============ Utilidades de limpieza ============

def _norm_region(s):
    """Normaliza nombres de región: mayúsculas y sin tildes."""
    if pd.isna(s):
        return s
    s2 = str(s).strip().upper()
    for a, b in zip("ÁÉÍÓÚ", "AEIOU"):
        s2 = s2.replace(a, b)
    return s2


def _norm_emp(x):
    """Limpia RUC/ID de empresa."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = s.replace("RUC", "").replace(":", "").replace(" ", "")
    return s if s else np.nan


def _clean_monto(x):
    """Normaliza montos a float, con varios formatos posibles."""
    if pd.isna(x):
        return 0.0
    s = str(x).strip().replace("S/", "").replace("s/", "").replace("soles", "").replace("SOLES", "").strip()
    # 437.479.13 -> 437479.13
    if s.count(".") > 1:
        parts = s.split(".")
        s = "".join(parts[:-1]) + "." + parts[-1]
    # 17800,50 -> 17800.50
    if "," in s and "." not in s:
        s = s.replace(",", ".")
    # 1,234,567.89 -> 1234567.89
    if s.count(",") > 0 and s.count(".") == 1:
        s = s.replace(",", "")
    s = s.replace(" ", "")
    try:
        return float(s)
    except Exception:
        t = "".join(re.findall(r"[0-9.,]", s))
        if t.count(".") > 1:
            parts = t.split(".")
            t = "".join(parts[:-1]) + "." + parts[-1]
        t = t.replace(",", "")
        try:
            return float(t)
        except Exception:
            return 0.0


## 2. Carga de datos y formato "long"

In [2]:
# ============ Carga ============

df = pd.read_excel(RUTA_ARCHIVO, sheet_name=HOJA_EXCEL)

df["FECHA"] = pd.to_datetime(df["FECHA"], dayfirst=True, errors="coerce")
df["ANIO"] = df["FECHA"].dt.year
df["REGION_NORM"] = df["REGION"].apply(_norm_region)

# Normalizar lista de regiones objetivo
REGIONES_NORM = [_norm_region(r) for r in REGIONES_OBJ]

# Columnas de empresas
emp_cols = [c for c in df.columns if str(c).upper().startswith("EMPRESA")]
if not emp_cols:
    raise ValueError("No se encontraron columnas EMPRESA_n en el archivo.")

for c in emp_cols:
    df[c] = df[c].apply(_norm_emp)

df["MONTO"] = df["MONTO"].apply(_clean_monto)

# Filtrar por regiones y años
df = df[
    df["REGION_NORM"].isin(REGIONES_NORM)
    & df["ANIO"].between(ANIO_MIN, ANIO_MAX)
].reset_index(drop=True)

if df.empty:
    raise ValueError("No hay registros para las regiones/años especificados.")

# ID de contrato y #empresas por contrato (para prorratear si quieres)
df["CONTRATO_ID"] = df.index.astype(str) + "_" + df["REGION_NORM"] + "_" + df["FECHA"].astype(str)
df["N_EMP_CONTRATO"] = df[emp_cols].notna().sum(axis=1).astype(int).replace(0, 1)

# ============ Formato largo: una fila por empresa-contrato ============

rows = []
for _, r in df.iterrows():
    anio = int(r["ANIO"])
    region = r["REGION_NORM"]
    monto = float(r["MONTO"])
    nemp = int(r["N_EMP_CONTRATO"])
    cid = r["CONTRATO_ID"]
    for c in emp_cols:
        e = r[c]
        if pd.notna(e) and str(e).strip():
            m = (monto / nemp) if PRORRATEAR_MONTO_CONSORCIO else monto
            rows.append((anio, region, str(e), m, cid))

long = pd.DataFrame(rows, columns=["ANIO", "REGION", "EMPRESA", "MONTO_CONTRATO", "CONTRATO_ID"])

# Agregado por año - región - empresa
edges = (
    long.groupby(["ANIO", "REGION", "EMPRESA"], as_index=False)
        .agg(
            monto_total=("MONTO_CONTRATO", "sum"),
            n_contratos=("CONTRATO_ID", "nunique")
        )
)

YEARS = sorted(edges["ANIO"].unique())
print("Años encontrados:", YEARS)


Años encontrados: [np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


## 3. Construir la red bipartita por año + métricas

In [3]:
# ============ Construcción de grafo por año (todas las regiones) ============

def build_graph_for_year(y):
    """
    Devuelve:
      - nodes: lista de nodos para vis.js / pyvis (con métricas en el tooltip)
      - edges_list: lista de aristas (peso = # contratos)
      - stats: métricas globales de la red ese año
    """
    dfa = edges[edges["ANIO"] == y].copy()
    if dfa.empty:
        return [], [], {
            "anio": int(y),
            "n_regiones": 0,
            "n_empresas": 0,
            "n_contratos": 0,
            "density": float("nan"),
        }

    # Grafo bipartito Región–Empresa
    G = nx.Graph()

    # Nodos de región
    for reg in dfa["REGION"].unique():
        G.add_node(reg, tipo="region", label=reg.title())

    # Nodos de empresa
    for emp in dfa["EMPRESA"].unique():
        G.add_node(emp, tipo="empresa", label=str(emp))

    # Aristas región–empresa, peso = #contratos
    for _, r in dfa.iterrows():
        reg = r["REGION"]
        emp = r["EMPRESA"]
        w = int(r["n_contratos"])
        monto_tot = float(r["monto_total"])
        if G.has_edge(reg, emp):
            # por si hubiera duplicados raros
            G[reg][emp]["weight"] += w
            G[reg][emp]["monto_total"] += monto_tot
        else:
            G.add_edge(reg, emp, weight=w, monto_total=monto_tot)

    # ------- Métricas por nodo -------
    # degree simple (número de vecinos)
    deg = dict(G.degree())
    # degree ponderado (# total de contratos)
    deg_w = dict(G.degree(weight="weight"))
    # betweenness y closeness
    bet = nx.betweenness_centrality(G, weight="weight", normalized=True)
    clo = nx.closeness_centrality(G)  # usamos versión no ponderada (para evitar ruido raro)

    # ------- Métricas globales -------
    total_contr = int(dfa["n_contratos"].sum())
    n_empresas = int(dfa["EMPRESA"].nunique())
    n_regiones = int(dfa["REGION"].nunique())
    dens = float(nx.density(G)) if G.number_of_nodes() > 1 else float("nan")

    stats = {
        "anio": int(y),
        "n_regiones": n_regiones,
        "n_empresas": n_empresas,
        "n_contratos": total_contr,
        "density": dens,
    }

    # ------- Preparar nodos para HTML (con métricas en tooltip) -------
    nodes = []
    for n in G.nodes():
        tipo = G.nodes[n]["tipo"]
        label = G.nodes[n]["label"]

        # tamaño y color
        if tipo == "region":
            size = 40
            color = "#1f77b4"
        else:
            # tamaño según degree ponderado (número de contratos)
            size = 8 + 2 * deg_w.get(n, 1)
            color = "#6baed6"

        title = (
            f"{'Región' if tipo=='region' else 'Empresa'}: {label}<br>"
            f"Degree: {deg.get(n, 0)}<br>"
            f"Degree ponderado (#contratos): {deg_w.get(n, 0)}<br>"
            f"Betweenness: {bet.get(n, 0):.3f}<br>"
            f"Closeness: {clo.get(n, 0):.3f}"
        )

        nodes.append({
            "id": n,
            "label": label,
            "shape": "circle" if tipo == "region" else "dot",
            "size": size,
            "color": color,
            "title": title,
            "group": tipo,
        })

    # ------- Preparar aristas para HTML -------
    edges_list = []
    for u, v, data in G.edges(data=True):
        w = int(data.get("weight", 1))
        monto_tot = float(data.get("monto_total", 0.0))
        width = 1 + 1.5 * w
        title = f"# contratos: {w} | Monto total: S/ {monto_tot:,.2f}"
        edges_list.append({
            "from": u,
            "to": v,
            "value": w,
            "width": width,
            "title": title,
        })

    return nodes, edges_list, stats


## 4. Fallback HTML con vis-network

In [4]:
# ============ Fallback HTML (vis-network) ============

def _write_visnetwork_html(nodes, edges_list, outfile_path, title_text):
    options = {
        "nodes": {"borderWidth": 1, "shape": "dot"},
        "edges": {"smooth": {"type": "dynamic"}},
        "interaction": {"hover": True, "tooltipDelay": 100},
        "physics": {"stabilization": {"iterations": 200}},
    }
    html = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>{title_text}</title>
<style>
body{{margin:0;background:#ffffff;font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif}}
#title{{padding:10px 12px;border-bottom:1px solid #eee}}
#network{{width:100vw;height:calc(100vh - 48px);}}
</style>
<link rel="preconnect" href="https://unpkg.com">
<script src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
</head>
<body>
<div id="title">{title_text}</div>
<div id="network"></div>
<script>
const nodes = new vis.DataSet({json.dumps(nodes, ensure_ascii=False)});
const edges = new vis.DataSet({json.dumps(edges_list, ensure_ascii=False)});
const container = document.getElementById('network');
const data = {{ nodes, edges }};
const options = {json.dumps(options, ensure_ascii=False)};
const network = new vis.Network(container, data, options);
</script>
</body>
</html>"""
    Path(outfile_path).write_text(html, encoding="utf-8")


## 5. Guardar HTML por año y recolectar métricas

In [5]:
# ============ Guardado por año: PyVis con fallback ============

def save_year_html(y, folder=SALIDA):
    dfa = edges[edges["ANIO"] == y].copy()
    if dfa.empty:
        return None

    nodes, edges_list, stats = build_graph_for_year(y)
    title_text = f"Red de contratación — Regiones {', '.join(REGIONES_OBJ)} {y}"

    folder.mkdir(parents=True, exist_ok=True)
    html_name = f"regiones_{y}.html"
    out_path = str(folder / html_name)

    # Intento con PyVis
    try:
        from pyvis.network import Network

        net = Network(
            height="720px",
            width="100%",
            bgcolor="#ffffff",
            font_color="#222",
            directed=False,
            notebook=False,
        )
        net.barnes_hut(
            gravity=-3500,
            central_gravity=0.12,
            spring_length=170,
            spring_strength=0.01,
            damping=0.9,
        )

        # Nodos
        for n in nodes:
            net.add_node(
                n["id"],
                label=n["label"],
                shape=n["shape"],
                size=n["size"],
                color=n["color"],
                title=n["title"],
                group=n.get("group"),
            )

        # Aristas
        for e in edges_list:
            net.add_edge(
                e["from"],
                e["to"],
                value=e["value"],
                width=e["width"],
                title=e["title"],
            )

        net.set_options("""
{
  "nodes": { "borderWidth": 1, "shape": "dot" },
  "edges": { "smooth": { "type": "dynamic" } },
  "interaction": { "hover": true, "tooltipDelay": 100 },
  "physics": { "stabilization": { "iterations": 200 } }
}
""")
        net.write_html(out_path, notebook=False, open_browser=False)
        return html_name, stats

    except Exception:
        # Fallback a vis-network puro
        _write_visnetwork_html(nodes, edges_list, out_path, title_text)
        return html_name, stats


# ============ Generar todos los años + recolectar stats para el index ============

items_np = []
for y in YEARS:
    res = save_year_html(int(y))
    if res:
        fname, st = res
        # asegurar tipos nativos para JSON
        dens_val = None
        if pd.notna(st["density"]):
            try:
                dens_val = float(st["density"])
            except Exception:
                dens_val = None

        items_np.append({
            "y": int(st["anio"]),
            "f": str(fname),
            "density": dens_val,
            "n_regiones": int(st["n_regiones"]),
            "n_empresas": int(st["n_empresas"]),
            "n_contratos": int(st["n_contratos"]),
        })

print(f"Generados {len(items_np)} HTMLs en: {SALIDA.resolve()}")

SALIDA.mkdir(parents=True, exist_ok=True)
index_html = SALIDA / "index.html"

items_js = json.dumps(items_np, ensure_ascii=False)


Generados 20 HTMLs en: C:\Users\aaro\OneDrive - APOYO COMUNICACION\Documentos\GitHub\Tesis_Redes\Redes_Ancash_Pasco\docs


## 6. Index final

In [6]:
# ============ Plantilla del index con placeholder para items ============

index_code = """<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, initial-scale=1"/>
<title>Redes de contratación - Regiones seleccionadas</title>
<style>
body{font-family:system-ui,-apple-system,Segoe UI,Roboto,Arial,sans-serif;margin:0;padding:18px;background:#fafafa}
.container{max-width:1200px;margin:0 auto}
h1{font-size:20px;margin:0 0 14px}
.bar{display:flex;gap:10px;align-items:center;flex-wrap:wrap;margin-bottom:12px}
label,select,button,small,span{font-size:14px}
iframe{width:100%;height:78vh;border:1px solid #e3e3e3;border-radius:10px;background:#fff}
button{padding:6px 10px;border:1px solid #ddd;border-radius:8px;background:#fff;cursor:pointer}
button:hover{background:#f2f2f2}
.kpi{background:#fff;border:1px solid #e7e7e7;border-radius:8px;padding:6px 10px}
.kpi b{font-weight:600}
small{color:#666}
</style>
</head>
<body>
<div class="container">
  <h1>Redes de contratación • Regiones seleccionadas</h1>
  <div class="bar">
    <button onclick="prev()">⟨ Anterior</button>
    <label for="year">Año:</label>
    <select id="year" onchange="go(this.value)"></select>
    <button onclick="next()">Siguiente ⟩</button>
    <span class="kpi" id="kpi">—</span>
    <small>
      Nodos azules grandes = regiones; puntos celestes = empresas.
      Grosor/tamaño dependen del # de contratos.
      Al pasar el mouse se muestran Degree, Betweenness y Closeness de cada nodo.
    </small>
  </div>
  <iframe id="view" src="" loading="lazy"></iframe>
</div>
<script>
const items = __ITEMS__;
let idx = 0;

function fillSelect() {
  const sel = document.getElementById('year');
  sel.innerHTML = "";
  items.forEach((it, i) => {
    const o = document.createElement('option');
    o.value = i;
    o.textContent = it.y;
    sel.appendChild(o);
  });
}

function updateKPI() {
  const it = items[idx];
  const dens = (it.density !== null && !isNaN(it.density)) ? Number(it.density).toFixed(4) : "—";
  document.getElementById('kpi').innerHTML =
    `<b>Densidad red:</b> ${dens} | ` +
    `<b>Regiones:</b> ${it.n_regiones} | ` +
    `<b>Empresas:</b> ${it.n_empresas} | ` +
    `<b>Contratos:</b> ${it.n_contratos}`;
}

function load() {
  if (!items.length) return;
  const it = items[idx];
  document.getElementById('view').src = it.f;
  document.getElementById('year').selectedIndex = idx;
  updateKPI();
}

function go(i) { idx = parseInt(i); load(); }
function prev() { idx = (idx - 1 + items.length) % items.length; load(); }
function next() { idx = (idx + 1) % items.length; load(); }

fillSelect();
load();
</script>
</body>
</html>"""

# Inserta el JSON ya serializado
index_code = index_code.replace("__ITEMS__", items_js)

index_html.write_text(index_code, encoding="utf-8")
print("✅ Index generado:", index_html.resolve())


✅ Index generado: C:\Users\aaro\OneDrive - APOYO COMUNICACION\Documentos\GitHub\Tesis_Redes\Redes_Ancash_Pasco\docs\index.html
